In [ ]:
#connectdatabase
import psycopg2

def connect_to_db():
    try:
        conn = psycopg2.connect(
            host='localhost',           # PostgreSQL host
            database='reg_ftu',           # PostgreSQL database name
            user='postgres',            # PostgreSQL username
            password='admin',           # PostgreSQL password
            port='5432'                 # PostgreSQL default port
        )
        return conn, conn.cursor()
    except Exception as e:
        print(f"Database connection error: {e}")
        exit()

conn, cursor = connect_to_db()

In [3]:
def create_tables():
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Users (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        username VARCHAR(255) UNIQUE NOT NULL,
        password VARCHAR(255) NOT NULL,
        role VARCHAR(10) CHECK (role IN ('Teacher', 'Student')) NOT NULL
    );
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Subjects (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        description TEXT,
        schedule VARCHAR(255),
        teacher_id INT REFERENCES Users(id) ON DELETE SET NULL
    );
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Enrollments (
        id SERIAL PRIMARY KEY,
        student_id INT REFERENCES Users(id) ON DELETE CASCADE,
        subject_id INT REFERENCES Subjects(id) ON DELETE CASCADE,
        grade NUMERIC(4, 2)
    );
    ''')

    conn.commit()
    print("Tables created successfully.")

create_tables()

Tables created successfully.


In [4]:
# Registration
def register_user():
    name = input("Enter your name: ")
    username = input("Create a username: ")
    password = input("Create a password: ")

    while True:
        role = input("Enter your role (Teacher/Student): ").capitalize()
        if role in ['Teacher', 'Student']:
            break
        else:
            print("Invalid role. Please enter 'Teacher' or 'Student'.")

    try:
        cursor.execute('INSERT INTO Users (name, username, password, role) VALUES (%s, %s, %s, %s)', (name, username, password, role))
        conn.commit()
        print(f"{role} registered successfully.")
    except errors.UniqueViolation:
        conn.rollback()
        print("Username already exists. Please choose another.")

In [5]:
# Login
def login():
    username = input("Username: ")
    password = input("Password: ")

    cursor.execute('SELECT * FROM Users WHERE username = %s AND password = %s', (username, password))
    user = cursor.fetchone()

    if user:
        print(f"\nWelcome {user[1]}! Logged in as {user[4]}.")
        if user[4] == 'Teacher':
            teacher_menu(user)
        elif user[4] == 'Student':
            student_menu(user)
    else:
        print("Invalid credentials.")

In [6]:
# Teacher Menu
def teacher_menu(user):
    while True:
        print("\n--- Teacher Menu ---")
        print("1. View Subjects and Students")
        print("2. Add Subject")
        print("3. Add/Edit Student Grades")
        print("4. View Teaching Schedule")
        print("5. Logout")
        choice = input("Select an option: ")

        if choice == '1':
            cursor.execute('SELECT id, name, description, schedule FROM Subjects WHERE teacher_id = %s', (user[0],))
            subjects = cursor.fetchall()
            if not subjects:
                print("No subjects found.")
            else:
                for subj in subjects:
                    print(f"\nSubject: {subj[1]}\nDescription: {subj[2]}\nSchedule: {subj[3]}")
                    cursor.execute('''
                        SELECT u.name FROM Enrollments e
                        JOIN Users u ON e.student_id = u.id
                        WHERE e.subject_id = %s
                    ''', (subj[0],))
                    students = cursor.fetchall()
                    print("Students:")
                    for student in students:
                        print(f"- {student[0]}")

        elif choice == '2':
            name = input("Subject Name: ")
            description = input("Subject Description: ")
            schedule = input("Schedule (e.g., Monday 13.00-15.00): ")
            cursor.execute('INSERT INTO Subjects (name, description, schedule, teacher_id) VALUES (%s, %s, %s, %s)', (name, description, schedule, user[0]))
            conn.commit()
            print("Subject added successfully.")

        elif choice == '3':
            cursor.execute('SELECT s.id, s.name FROM Subjects s WHERE s.teacher_id = %s', (user[0],))
            subjects = cursor.fetchall()
            if not subjects:
                print("No subjects found.")
                continue
            print("Subjects:")
            for subj in subjects:
                print(f"{subj[0]}: {subj[1]}")
            subject_id = int(input("Enter subject ID to edit grades: "))

            cursor.execute('''
                SELECT e.id, u.name, e.grade FROM Enrollments e
                JOIN Users u ON e.student_id = u.id
                WHERE e.subject_id = %s
            ''', (subject_id,))
            enrollments = cursor.fetchall()
            if not enrollments:
                print("No students enrolled in this subject.")
                continue
            for enrollment in enrollments:
                print(f"{enrollment[0]}: {enrollment[1]} - Grade: {enrollment[2]}")
                choice = input("Edit grade? (y/n): ")
                if choice.lower() == 'y':
                    new_grade = float(input("Enter new grade: "))
                    cursor.execute('UPDATE Enrollments SET grade = %s WHERE id = %s', (new_grade, enrollment[0]))
                    conn.commit()
                    print("Grade updated.")

        elif choice == '4':
            cursor.execute('SELECT schedule, name FROM Subjects WHERE teacher_id = %s ORDER BY schedule', (user[0],))
            schedule = cursor.fetchall()
            if not schedule:
                print("No schedule available.")
            else:
                print("\nTeaching Schedule:")
                for sch in schedule:
                    print(f"{sch[0]} ({sch[1]})")

        elif choice == '5':
            print("Logged out.")
            break
        else:
            print("Invalid option.")

In [7]:
# Student Menu
def student_menu(user):
    while True:
        print("\n--- Student Menu ---")
        print("1. View Enrolled Subjects")
        print("2. View Grades")
        print("3. View Class Schedule")
        print("4. Logout")
        choice = input("Select an option: ")

        if choice == '1':
            cursor.execute('''
                SELECT s.name, s.description, s.schedule FROM Subjects s
                JOIN Enrollments e ON s.id = e.subject_id
                WHERE e.student_id = %s
            ''', (user[0],))
            subjects = cursor.fetchall()
            if subjects:
                for subj in subjects:
                    print(f"\nSubject: {subj[0]}\nDescription: {subj[1]}\nSchedule: {subj[2]}")
            else:
                print("No enrolled subjects.")

        elif choice == '2':
            cursor.execute('''
                SELECT s.name, e.grade FROM Subjects s
                JOIN Enrollments e ON s.id = e.subject_id
                WHERE e.student_id = %s
            ''', (user[0],))
            grades = cursor.fetchall()
            if grades:
                print("\nYour Grades:")
                for subj, grade in grades:
                    print(f"{subj} - Grade: {grade}")
            else:
                print("No grades available.")

        elif choice == '3':
            cursor.execute('''
                SELECT s.schedule, s.name FROM Subjects s
                JOIN Enrollments e ON s.id = e.subject_id
                WHERE e.student_id = %s ORDER BY schedule
            ''', (user[0],))
            schedule = cursor.fetchall()
            if schedule:
                print("\nClass Schedule:")
                for sch in schedule:
                    print(f"{sch[0]} ({sch[1]})")
            else:
                print("No schedule available.")

        elif choice == '4':
            print("Logged out.")
            break
        else:
            print("Invalid option.")

In [ ]:
# Main Menu
def main_menu():
    while True:
        print("\n--- Welcome to School Management System ---")
        print("1. Register")
        print("2. Log in")
        print("3. Exit")
        choice = input("Select an option: ")

        if choice == '1':
            register_user()
        elif choice == '2':
            login()
        elif choice == '3':
            print("Thank you for using the system. Goodbye!")
            break
        else:
            print("Invalid option.")

if __name__ == "__main__":
    try:
        main_menu()
    except Exception as e:
        print(f"Error: {e}")
    finally:
        cursor.close()
        conn.close()


--- Welcome to School Management System ---
1. Register
2. Log in
3. Exit
